In [3]:
import skimage
from skimage.io import imread
from glob import glob
from Crypto.Cipher import AES, DES
import random
import string
from PIL import Image

In [7]:
#Randomly generate 16 strings composed of lowercase letters
def key_generator(size = 16, chars = string.ascii_lowercase):
    return ''.join(random.choice(chars) for _ in range(size))

def pad(data):
    return data + b"\x00" * (16 - len(data) % 16)


def aes_cbc_encrypt(key, data, mode=AES.MODE_CBC):
    #IV is a random value
    IV = key_generator(16)
    aes = DES.new(key.encode("utf8"), mode, IV.encode("utf8"))
    new_data = aes.encrypt(data)
    return new_data

def aes_ecb_encrypt(key, data, mode=AES.MODE_ECB):
    aes = AES.new(key.encode("utf8"), mode)
    new_data = aes.encrypt(data)
    return new_data

def des_cbc_encrypt(key, data, mode=DES.MODE_CBC):
    #IV is a random value
    IV = key_generator(8)
    aes = DES.new(key.encode("utf8"), mode, IV.encode("utf8"))
    new_data = aes.encrypt(data)
    return new_data

def des_ecb_encrypt(key, data, mode=DES.MODE_ECB):
    aes = DES.new(key.encode("utf8"), mode)
    new_data = aes.encrypt(data)
    return new_data


def img_aes(ori_img):
    byte_img = ori_img.tobytes()
    imlength = len(byte_img)
    key = key_generator(8)

    value_encrypt = des_cbc_encrypt(key, pad(byte_img))[:imlength]

    im2 = Image.new('L', (112,92))
    im2.putdata(value_encrypt)

    # Save the object as an image in the corresponding format
    return im2


for idx in range(40):
    ori_img_path = glob('AT&T/s{}/*.pgm'.format(idx))
    
    for p in ori_img_path:
        ori_img = imread(p)
        im_enc = img_aes(ori_img)
        im_enc.save(p.replace('AT&T','AT&T_AES') , "PPM")